In [1]:
import pandas as pd
import numpy as np

In [2]:
df_index= pd.read_excel('C:/Users/XLL/Desktop/数据/CSI300_Index & Future_0521Final/CSI300_Index & Future/CSI300_Index_Level.xlsx')
df_index.head()

,date,index_level
0,2010-04-16,3356.332
1,2010-04-19,3176.423
2,2010-04-20,3173.374
3,2010-04-21,3236.679
4,2010-04-22,3201.541


In [3]:
df_2008 = pd.read_excel('C:/Users/XLL/Desktop/数据/CSI300_Index & Future_0521Final/CSI300_Index & Future/CSI300_Index_Level(2008-3-31).xlsx')
df_2008.head()



,指标名称,沪深300指数
0,频率,日
1,单位,点
2,指标ID,M002845714
3,2022-12-30,3871.63
4,2022-12-29,3856.7


In [4]:
a = ['2010-04-15',3394.57]
df_index2 = pd.DataFrame(np.insert(df_index.values,0,a,axis = 0))
df_index2.columns = df_index.columns
df_index2

,date,index_level
0,2010-04-15,3394.57
1,2010-04-16,3356.332
2,2010-04-19,3176.423
3,2010-04-20,3173.374
4,2010-04-21,3236.679
...,...,...
3087,2022-12-26,3843.4887
3088,2022-12-27,3887.854
3089,2022-12-28,3871.2644
3090,2022-12-29,3856.7007


In [5]:
diff = np.array(df_index2["index_level"])
diff = np.diff(diff)
diff

array([-38.238000000000284, -179.9090000000001, -3.048999999999978, ...,
       -16.58959999999979, -14.563700000000154, 14.933100000000195],
      dtype=object)

In [6]:
df_index3 = pd.DataFrame(np.insert(df_index.values,2,diff,axis = 1))
df_index3.columns = ['date','index_level','diff']
df_index3['date']=pd.to_datetime(df_index3['date'])
df_index3

,date,index_level,diff
0,2010-04-16,3356.332,-38.238
1,2010-04-19,3176.423,-179.909
2,2010-04-20,3173.374,-3.049
3,2010-04-21,3236.679,63.305
4,2010-04-22,3201.541,-35.138
...,...,...,...
3086,2022-12-26,3843.4887,15.2699
3087,2022-12-27,3887.854,44.3653
3088,2022-12-28,3871.2644,-16.5896
3089,2022-12-29,3856.7007,-14.5637


In [7]:
df_index3.set_index("date", inplace=True)
df_index3


,index_level,diff
date,,
2010-04-16,3356.332,-38.238
2010-04-19,3176.423,-179.909
2010-04-20,3173.374,-3.049
2010-04-21,3236.679,63.305
2010-04-22,3201.541,-35.138
...,...,...
2022-12-26,3843.4887,15.2699
2022-12-27,3887.854,44.3653
2022-12-28,3871.2644,-16.5896


In [8]:
df_index4 = df_index3.drop('index_level',axis = 1)
df_index4
#check missing value
df_index4.isnull().sum()
#find out missing value
df_index4[df_index4.isnull().values==True]
df_index4.dropna(inplace=True)
df_index4

,diff
date,
2010-04-16,-38.238
2010-04-19,-179.909
2010-04-20,-3.049
2010-04-21,63.305
2010-04-22,-35.138
...,...
2022-12-26,15.2699
2022-12-27,44.3653
2022-12-28,-16.5896


In [9]:
df_index4['binary'] = df_index4['diff'].apply(lambda x: 1 if x > 0 else 0)
df_index4

,diff,binary
date,,
2010-04-16,-38.238,0
2010-04-19,-179.909,0
2010-04-20,-3.049,0
2010-04-21,63.305,1
2010-04-22,-35.138,0
...,...,...
2022-12-26,15.2699,1
2022-12-27,44.3653,1
2022-12-28,-16.5896,0


In [10]:
quarterly_groups=df_index4.groupby([pd.Grouper(freq='Q'),'binary']).mean()
quarterly_groups

diff
date       binary           
2010-06-30 0      -46.287500
           1       36.094444
2010-09-30 0      -29.649600
           1       29.309053
2010-12-31 0      -40.251700
...                      ...
2022-06-30 1       45.621215
2022-09-30 0      -32.008532
           1       26.342679
2022-12-31 0      -32.631187
           1       37.183286

[102 rows x 1 columns]

In [11]:
df = quarterly_groups
df

diff
date       binary           
2010-06-30 0      -46.287500
           1       36.094444
2010-09-30 0      -29.649600
           1       29.309053
2010-12-31 0      -40.251700
...                      ...
2022-06-30 1       45.621215
2022-09-30 0      -32.008532
           1       26.342679
2022-12-31 0      -32.631187
           1       37.183286

[102 rows x 1 columns]

In [12]:
grouped = df.groupby('date')['diff']
diff1 = grouped.apply(lambda x: x.loc[x.index.get_level_values('binary') == 1].sum() / x.loc[x.index.get_level_values('binary') == 0].sum() if 0 in x.index.get_level_values('binary') and 1 in x.index.get_level_values('binary') else '+' if 1 in x.index.get_level_values('binary') else '-').to_numpy()
diff1

array([-0.77978816, -0.98851427, -1.12216298, -0.89845664, -0.82756804,
       -0.83343581, -0.97952822, -1.0664223 , -1.28223824, -0.79932418,
       -1.39541608, -0.83712534, -0.88430223, -1.22946503, -1.11160151,
       -0.95477646, -1.23151263, -1.17580323, -1.72356963, -0.92873936,
       -0.83610605, -0.69538223, -1.04235744, -0.53566953, -0.9383016 ,
       -1.11722881, -1.10743615, -1.0415918 , -1.33457313, -1.11895865,
       -0.93181699, -0.5259112 , -0.95705394, -1.28793624, -0.96731545,
       -1.65340168, -1.10010445, -0.95822155, -1.28245567, -0.77205845,
       -1.09713453, -0.93531263, -1.43063121, -0.80321222, -0.89359918,
       -0.88479099, -0.97911288, -0.79892292, -0.95419906, -0.8229893 ,
       -1.13950149])

In [13]:
diff2 = list(diff1)
diff2

[-0.7797881597503519,
 -0.9885142676993596,
 -1.1221629829779978,
 -0.8984566434025765,
 -0.8275680430853111,
 -0.8334358116838645,
 -0.9795282161834793,
 -1.0664222954348905,
 -1.282238236242221,
 -0.7993241795861498,
 -1.3954160819937946,
 -0.8371253437377053,
 -0.8843022297475005,
 -1.2294650291349039,
 -1.1116015128249779,
 -0.9547764563432123,
 -1.2315126321293755,
 -1.1758032343093268,
 -1.7235696292596605,
 -0.9287393621109296,
 -0.8361060538277713,
 -0.695382230414032,
 -1.0423574435926235,
 -0.5356695294160115,
 -0.9383016012904071,
 -1.1172288062530156,
 -1.1074361452374164,
 -1.0415918049364383,
 -1.3345731268303282,
 -1.1189586503405282,
 -0.9318169887939048,
 -0.5259112001763777,
 -0.9570539350314559,
 -1.2879362368422123,
 -0.9673154526189794,
 -1.6534016818116353,
 -1.1001044541659457,
 -0.9582215514194081,
 -1.2824556700627232,
 -0.7720584478334759,
 -1.0971345326679265,
 -0.9353126282940963,
 -1.4306312107420214,
 -0.8032122248534819,
 -0.8935991817808808,
 -0.88479099

In [14]:
diff3 = []
for i in diff2:
    if i != '+' and i != '-':
        i = abs(float(i))
        i = 100*(1-1/(1+i))
    diff3.append(i)
diff3

[43.81353789092134,
 49.71119814206993,
 52.8782658061109,
 47.32563403672394,
 45.28247504745191,
 45.4575942267878,
 49.482912553376224,
 51.607181058335215,
 56.18336490380897,
 44.42357795525189,
 58.25359913390653,
 45.56713272674907,
 46.929957189829274,
 55.146190367111124,
 52.6425798652624,
 48.84325536277977,
 55.18734755958919,
 54.03996169178259,
 63.28347954622233,
 48.152662840585194,
 45.53691504282785,
 41.01625096331295,
 51.03697429961414,
 34.88182314978388,
 48.40844173402845,
 52.768449161158024,
 52.54897747388956,
 51.01861216419149,
 57.16561676704857,
 52.80700735527357,
 48.235262149530435,
 34.46538698422218,
 48.902787904824564,
 56.292488230345505,
 49.1693109679612,
 62.31252859848795,
 52.38332083833663,
 48.933255316531756,
 56.18753901264074,
 43.568452766182574,
 52.31588701523011,
 48.328761700818255,
 58.858423458870966,
 44.54341057491145,
 47.19051372532142,
 46.943719293564044,
 49.47231112717033,
 44.41118117704711,
 48.82814048615642,
 45.145042

In [15]:
diff4 = []
for i in diff3:
    if i == '-':
        i = int('0')
    if i =='+':
        i = int('100')
    diff4.append(i)
diff4

[43.81353789092134,
 49.71119814206993,
 52.8782658061109,
 47.32563403672394,
 45.28247504745191,
 45.4575942267878,
 49.482912553376224,
 51.607181058335215,
 56.18336490380897,
 44.42357795525189,
 58.25359913390653,
 45.56713272674907,
 46.929957189829274,
 55.146190367111124,
 52.6425798652624,
 48.84325536277977,
 55.18734755958919,
 54.03996169178259,
 63.28347954622233,
 48.152662840585194,
 45.53691504282785,
 41.01625096331295,
 51.03697429961414,
 34.88182314978388,
 48.40844173402845,
 52.768449161158024,
 52.54897747388956,
 51.01861216419149,
 57.16561676704857,
 52.80700735527357,
 48.235262149530435,
 34.46538698422218,
 48.902787904824564,
 56.292488230345505,
 49.1693109679612,
 62.31252859848795,
 52.38332083833663,
 48.933255316531756,
 56.18753901264074,
 43.568452766182574,
 52.31588701523011,
 48.328761700818255,
 58.858423458870966,
 44.54341057491145,
 47.19051372532142,
 46.943719293564044,
 49.47231112717033,
 44.41118117704711,
 48.82814048615642,
 45.145042

In [16]:
df_index5=df_index4.groupby([pd.Grouper(freq='Q')]).mean()
df_index5
df_index5.dropna(inplace=True)
df_index5['RSI'] = diff4
df_index5 = df_index5.drop('binary',axis =1)
df_index5

,RSI
date,
2010-06-30,43.813538
2010-09-30,49.711198
2010-12-31,52.878266
2011-03-31,47.325634
2011-06-30,45.282475
2011-09-30,45.457594
2011-12-31,49.482913
2012-03-31,51.607181
2012-06-30,56.183365


In [17]:
df_index5.to_csv('C:/Users/XLL/Desktop/数据/CSI300_Index & Future_0521Final/CSI300_Index & Future/Quarterly_RSI.csv', index = False, header=True)